In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [9]:
original = pd.read_csv('airlinedelaycauses_DelayedFlights.csv')

In [10]:
original.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = original.copy()

In [12]:
data = data.rename(columns={'Unnamed: 0': 'ID'})

In [19]:
data['DayofMonth'].max()

31

In [22]:
# merge Year, Month, DayofMonth and  DayOfWeek into a single Date column
data['Date'] = pd.to_datetime(dict(
    year=data['Year'],
    month=data['Month'],
    day=data['DayofMonth']
))


In [24]:
data.drop(columns=['Year', 'Month', 'DayofMonth'], inplace=True)

In [28]:
data.rename(columns={'CRSDepTime': 'ScheduledDepTime', 'DepTime' : 'ActualDepTime', 'CRSArrTime': 'ScheduledArrTime', 'ArrTime': 'ActualArrTime', 'CRSElapsedTime' : 'ScheduledElapsedTime'}, inplace=True)

In [45]:
data['CancellationReason'] = data['CancellationCode'].map({
    'N': 'Not Cancelled',
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'NAS'
})
data.drop(columns=['CancellationCode'], inplace=True)

In [50]:
data.columns

Index(['ID', 'DayOfWeek', 'ActualDepTime', 'ScheduledDepTime', 'ActualArrTime',
       'ScheduledArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'ScheduledElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay', 'Date', 'CancellationReason'],
      dtype='object')

In [55]:
data['Diverted']

0          0
1          0
2          0
3          0
4          0
          ..
1936753    0
1936754    0
1936755    0
1936756    0
1936757    0
Name: Diverted, Length: 1936758, dtype: int64

In [61]:
data['ScheduledDepTime']

0          1955
1           735
2           620
3          1755
4          1915
           ... 
1936753    1220
1936754     600
1936755     847
1936756    1240
1936757    1103
Name: ScheduledDepTime, Length: 1936758, dtype: int64

In [ ]:
for col in ['ActualDepTime', 'ScheduledDepTime', 'ActualArrTime', 'ScheduledArrTime']:
    times = (
        data[col]
        .fillna(0)
        .astype(int)
        .astype(str)
        .str.zfill(4)
        .replace('2400', '0000')
    )
    data[col] = pd.to_datetime(times, format='%H%M', errors='coerce').dt.time


In [69]:
data['Date'] = pd.to_datetime(data['Date'])

In [72]:
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest', 'CancellationReason', 'DayOfWeek']
data[categorical_cols] = data[categorical_cols].astype('category')

In [76]:
data['Cancelled'] = data['Cancelled'].astype(bool)
data['Diverted'] = data['Diverted'].astype(bool)

In [ ]:
delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
data[delay_cols] = data[delay_cols].fillna(0)

In [109]:
data.isnull().sum()

ID                         0
DayOfWeek                  0
ActualDepTime              0
ScheduledDepTime           0
ActualArrTime              0
ScheduledArrTime           0
UniqueCarrier              0
FlightNum                  0
TailNum                    5
ActualElapsedTime       8387
ScheduledElapsedTime     198
AirTime                 8387
ArrDelay                8387
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                  7110
TaxiOut                  455
Cancelled                  0
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
Date                       0
CancellationReason         0
dtype: int64

In [77]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 28 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   DayOfWeek             category      
 2   ActualDepTime         object        
 3   ScheduledDepTime      object        
 4   ActualArrTime         object        
 5   ScheduledArrTime      object        
 6   UniqueCarrier         category      
 7   FlightNum             int64         
 8   TailNum               object        
 9   ActualElapsedTime     float64       
 10  ScheduledElapsedTime  float64       
 11  AirTime               float64       
 12  ArrDelay              float64       
 13  DepDelay              float64       
 14  Origin                category      
 15  Dest                  category      
 16  Distance              int64         
 17  TaxiIn                float64       
 18  TaxiOut               float64       
 19  

In [112]:
data.to_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv', index=False)

In [113]:
data_cleaned = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Visualizations

In [152]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

data = data_cleaned.copy()
# Preprocess columns for visuals

# Convert ScheduledDepTime to numeric, invalid parsing becomes NaN
data['ScheduledDepTime'] = pd.to_numeric(data['ScheduledDepTime'], errors='coerce')

# Now you can get the Hour
data['Hour'] = (data['ScheduledDepTime'] // 100).fillna(0).astype(int)

data['DayOfWeek'] = data['DayOfWeek'].astype(int)

# Initialize Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Flight Delay Dashboard", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Select Delay Type:"),
        dcc.Dropdown(
            id='delay-type-dropdown',
            options=[
                {'label': 'Arrival Delay', 'value': 'ArrDelay'},
                {'label': 'Departure Delay', 'value': 'DepDelay'}
            ],
            value='ArrDelay'
        ),
        dcc.Graph(id='delay-over-time')
    ]),

    html.Div([dcc.Graph(id='airline-delay-bar')]),
    html.Div([dcc.Graph(id='heatmap-delay')]),
    html.Div([dcc.Graph(id='cancellation-pie')])
])

# Callbacks for interactivity
@app.callback(
    [Output('delay-over-time', 'figure'),
     Output('airline-delay-bar', 'figure'),
     Output('heatmap-delay', 'figure'),
     Output('cancellation-pie', 'figure')],
    [Input('delay-type-dropdown', 'value')]
)
def update_charts(selected_delay):
    # 1. Delay over time
    delay_time = data.groupby('Date')[selected_delay].mean().reset_index()
    fig_line = px.line(delay_time, x='Date', y=selected_delay,
                       title=f'Average {selected_delay} Over Time')

    # 2. Airline delay comparison
    airline_delay = data.groupby('UniqueCarrier')[selected_delay].mean().reset_index()
    fig_bar = px.bar(airline_delay, x='UniqueCarrier', y=selected_delay,
                     title=f'Average {selected_delay} per Airline', text=selected_delay)

    # 3. Heatmap: day of week vs hour
    heatmap_data = data.groupby(['DayOfWeek', 'Hour'])[selected_delay].mean().reset_index()
    fig_heatmap = px.density_heatmap(heatmap_data, x='Hour', y='DayOfWeek', z=selected_delay,
                                     nbinsx=24, nbinsy=7,
                                     title=f'Average {selected_delay} by Day of Week & Hour',
                                     labels={'Hour':'Hour of Day', 'DayOfWeek':'Day of Week'})

    # 4. Cancellation pie chart
    cancel_data = data[data['Cancelled'] == 1]['CancellationReason'].value_counts().reset_index()
    cancel_data.columns = ['Reason', 'Count']
    fig_pie = px.pie(cancel_data, names='Reason', values='Count', title='Cancellation Reasons')

    return fig_line, fig_bar, fig_heatmap, fig_pie

# Run server
if __name__ == '__main__':
    app.run(debug=True)
